## Pipeline Parameters

In [0]:
dbutils.widgets.text("input_number_of_days", '30')
dbutils.widgets.text("target_table_name", 'kdayno_gold_posts_last_30_days_overall_sentiment_agg')

input_number_of_days = dbutils.widgets.get("input_number_of_days")
target_table_name = dbutils.widgets.get("target_table_name")

In [0]:
# Standard library imports
import os

# Third-party library imports
from dotenv import load_dotenv

In [0]:
load_dotenv()

catalog_name = os.getenv('DATABRICKS_CATALOG_NAME')
schema_name = os.getenv('DATABRICKS_SCHEMA_NAME')

In [0]:
print(f"Running aggregate process for last: {input_number_of_days} days ")

reddit_posts_sentiment_agg_df = spark.sql(
    f"""
    WITH all_posts AS (
      SELECT 
        * 
        , COUNT(*) OVER(PARTITION BY ticker_symbol) AS total_posts
      FROM tabular.dataexpert.kdayno_silver_reddit_all_posts
      WHERE created_date_utc BETWEEN current_date() - INTERVAL {input_number_of_days} DAY AND current_date()
      ),

    posts_sentiment_agg AS (
      SELECT
          ticker_symbol
          , company_name
          , sentiment_category
          , COUNT(*) AS post_count
          , total_posts
          , ROUND(AVG(sentiment_score), 2) AS average_sentiment_score
          , ROUND(COUNT(*) / total_posts , 2) AS sentiment_category_percentage
      FROM all_posts
      GROUP BY ticker_symbol, company_name, sentiment_category, total_posts
      ),        

    posts_sentiment_ranked_agg AS (
      SELECT
        ticker_symbol
        , company_name
        , sentiment_category
        , ROW_NUMBER() OVER(PARTITION BY ticker_symbol ORDER BY sentiment_category_percentage DESC) AS sentiment_rank
        , post_count
        , total_posts
        , sentiment_category_percentage
        , average_sentiment_score
      FROM posts_sentiment_agg
      )

    SELECT *
    FROM posts_sentiment_ranked_agg
    ORDER BY ticker_symbol, sentiment_rank
    """)

In [0]:
print(f"Loading data to target table: {catalog_name}.{schema_name}.{target_table_name}")

(reddit_posts_sentiment_agg_df.write.format('delta')
                                .mode('overwrite')
                                .saveAsTable(f'{catalog_name}.{schema_name}.{target_table_name}'))